# Washington State DOT Traffic API

source: [https://wsdot.wa.gov/traffic/api](https://wsdot.wa.gov/traffic/api)


In [20]:
import pandas as pd
import json
import os
import requests

from dotenv import load_dotenv
from pathlib import Path
from sqlalchemy import create_engine

## Import env variables


In [21]:
# load the .env file
load_dotenv()

# API variables
api_key = os.getenv("API_KEY")

# MySQL database variables
db_user = os.getenv("AZURE_USERNAME")
db_pwd = os.getenv("AZURE_PWD")
db_host = os.getenv("AZURE_URL")
db_port = os.getenv("AZURE_PORT")
db_database = os.getenv("AZURE_DB")

# print(api_key)

## URLs to Access APIs


In [22]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

### Custom function to get API data


In [23]:
def get_api_data(url, access_key):
    """_summary_

    Args:
        url (_type_): _description_
        access_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code

In [24]:
def convert_dict_to_json(dataframe):

    for col in dataframe.columns:
        if dataframe[col].apply(lambda x: isinstance(x, dict)).all():
            dataframe[col] = dataframe[col].apply(lambda x: json.dumps(x))

In [25]:
def get_updates(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(traffic_alerts_url, access_key)
    df_wa, response_wa = get_api_data(weather_information_url, access_key)

    # add pull_date_time
    df_tt["timestamp"] = timestamp
    df_ta["timestamp"] = timestamp
    df_wa["timestamp"] = timestamp

    # convert dict columns to json
    convert_dict_to_json(df_tt)
    convert_dict_to_json(df_ta)
    convert_dict_to_json(df_wa)

    df_api_fetch = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
            }
        ]
    )

    return df_api_fetch, df_tt, df_ta, df_wa

# EXTRACT

## Call to get all info from API


In [26]:
df_api_fetch, df_tt, df_ta, df_wa = get_updates(
    TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.


# LOAD

## Export to CSV


In [27]:
# Set path to datasets folder
# datasets = Path("../datasets-2")
# df_api_fetch.to_csv(datasets / "api_fetch.csv", index=False)
# df_tt.to_csv(datasets / "travel_times.csv", index=False)
# df_ta.to_csv(datasets / "traffic_alerts.csv", index=False)
# df_wa.to_csv(datasets / "weather_alerts.csv", index=False)

## Connection to MySQL Database


In [28]:
# Connect to database
connection_url = (
    f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
)
try:
    engine = create_engine(connection_url)
    print("Connection successful!")
except Exception as e:
    print("Connection unsuccessful! The error is: ", e)

Connection successful!


## Upload Data to MySQL


In [42]:
def upload_to_db(dataframe, table_name):
    """_summary_

    Args:
        dataframe (_type_): _description_
        table_name (_type_): _description_
    """
    try:
        dataframe.to_sql(
            table_name,
            con=engine,
            if_exists="append",
            index=False,
        )
        print(f"{table_name} uploaded successfully!")
    except Exception as e:
        print(f"Failed to upload {table_name}! The error is: ", e)

In [31]:
# # Table api_fetch
# df_api_fetch.to_sql("api_fetch", con=engine, if_exists="append", index=False)

# # Table time_trave_raw
# df_tt.to_sql("time_travel_raw", con=engine, if_exists="append", index=False)

# # Table traffic_alerts_raw
# df_ta.to_sql("traffic_alerts_raw", con=engine, if_exists="append", index=False)

# # Table weather_alerts_raw
# df_wa.to_sql("weather_alerts_raw", con=engine, if_exists="append", index=False)

# print("Data saved to database.")

Data saved to database.


In [43]:
def extract_and_load():
    # Get data from the API
    df_api_fetch, df_tt, df_ta, df_wa = get_updates(
        TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
    )

    # Connect to database
    connection_url = (
        f"mysql+pymysql://{db_user}:{db_pwd}" f"@{db_host}:{db_port}/{db_database}"
    )
    try:
        engine = create_engine(connection_url)
        print("Connection successful!")
    except Exception as e:
        print("Connection unsuccessful! The error is: ", e)

    # Upload data to the database
    upload_to_db(df_api_fetch, "api_fetch")
    upload_to_db(df_tt, "time_travel_raw")
    upload_to_db(df_ta, "traffic_alerts_raw")
    upload_to_db(df_wa, "weather_alerts_raw")


Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!


In [44]:
import schedule
import time

schedule.every(2).minutes.do(extract_and_load)

while True:
    schedule.run_pending()
    time.sleep(60)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!
Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!
Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
weather_alerts_raw uploaded successfully!
Data fetched successfully.
Data fetched successfully.
Data fetched successfully.
Connection successful!
api_fetch uploaded successfully!
time_travel_raw uploaded successfully!
traffic_alerts_raw uploaded successfully!
we

KeyboardInterrupt: 

In [45]:
df_check = pd.read_sql("SELECT * FROM api_fetch", engine)
display(df_check)

,id,travel_times_response,traffic_alerts_response,weather_alerts_response,timestamp
0,21,200,200,200,2025-02-04 16:23:32
1,22,200,200,200,2025-02-04 16:50:07
2,23,200,200,200,2025-02-04 16:50:21
3,24,200,200,200,2025-02-04 16:50:24
4,25,200,200,200,2025-02-04 16:50:28
5,26,200,200,200,2025-02-04 16:53:16
6,27,200,200,200,2025-02-04 16:53:19
7,28,200,200,200,2025-02-04 16:56:10
8,29,200,200,200,2025-02-04 16:56:11
9,30,200,200,200,2025-02-04 16:56:14


In [19]:
df_schema = pd.read_sql("DESCRIBE weather_alerts_raw", engine)
display(df_schema)

,Field,Type,Null,Key,Default,Extra
0,id,int unsigned,NO,PRI,None,auto_increment
1,barometricpressure,float,YES,,None,
2,latitude,float,YES,,None,
3,longitude,float,YES,,None,
4,precipitationininches,float,YES,,None,
5,readingtime,varchar(45),YES,,None,
6,relativehumidity,float,YES,,None,
7,skycoverage,varchar(45),YES,,None,
8,stationid,varchar(45),YES,,None,
9,stationname,longtext,YES,,None,


In [44]:
cols = df_wa.columns
for col in cols:
    print(col)

BarometricPressure
Latitude
Longitude
PrecipitationInInches
ReadingTime
RelativeHumidity
SkyCoverage
StationID
StationName
TemperatureInFahrenheit
Visibility
WindDirection
WindDirectionCardinal
WindGustSpeedInMPH
WindSpeedInMPH
timestamp
wa_id
